In [1]:
import numpy as np
import pandas as pd
import pulp
from scipy.spatial import distance_matrix
import itertools
import matplotlib.pyplot as plt
%matplotlib inline 

In [4]:
customer_count = 11
vehicle_count = 7
vehicle_capacity = 50
p = 3
syoumi = range(1,4)
m = 2
c = 30
r = 3
C = 0.1
h = 0.06
f = 60
Z = 0.4
i_customer = range(0,12)
j_customer = range(0,12)
k_vehicle = range(1,10)
t_customer = range(1,4)
kikan = 4

#需要　dの設定
u = [
    [0,0,0,0],
    [900,400,1000,600],
    [1400,1200,17,1200],
    [500,500,1250,600],
    [1100,2500,500,400],
    [1050,900,1500,1100],
    [1200,500,400,1400],
    [800,700,500,500],
    [1900,400,300,1300],
    [800,400,700,1300],
    [1100,1600,400,300],
    [2600,3200,2500],
]

#距離aの設定
a = [
    [0,67,89.2,126,78.1,70.6,106,66.3,64.4,156,151,35.5],
    [73.2,0,154,191,143,144,141,101,74.3,166,176,61.5],
    [70.8,136,0,65.9,62.9,113,158,118,126,218,212,97.2],
    [126,192,69.5,0,98.9,171,233,193,182,274,268,153],
    [78.4,144,63.2,99.7,0,123,185,145,134,226,220,105],
    [70.9,144,105,163,115,0,50.2,58.4,120,155,220,105],
    [106,131,161,222,175,50.9,0,41.6,75.3,105,199,84.4],
    [66.5,91.2,121,182,135,58.2,40.1,0,35.3,117,159,44.4],
    [67.4,74.9,149,185,137,92.7,74.4,34.5,0,92.1,120,34.8],
    [158,166,239,276,228,155,106,116,92.4,0,69.6,126],
    [150,176,232,268,220,221,192,152,119,70,0,119],
    [35,60.3,116,153,105,106,83.6,43.7,30.6,123,118,0]
]



# 問題の宣言
problem = pulp.LpProblem("CVRP", pulp.LpMinimize)

#１１～１５で変数を定義している　目的関数から下は制約条件
#決定変数、経路を通るかどうか変数（１１）
x = [[[[pulp.LpVariable("x%s_%s,%s,%s"%(i,j,k,t), cat="Binary") if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)] for t in range(kikan)]

#変数(12)
F = [[[[pulp.LpVariable("F",(i,j,k,t)) if i != j else None for k in range(vehicle_count)]for j in range(customer_count)] for i in range(customer_count)] for t in range(kikan)]

#変数（１３）
I = pulp.LpVariable.dicts("zaiko",(i_customer,t_customer))

#変数(14)
Ip = pulp.LpVariable.dicts("zaikopurasu",(i_customer,t_customer),lowBound =0)
W = pulp.LpVariable.dicts("haiki",(i_customer,j_customer),lowBound =0)

#変数(15)
Q = [[[pulp.LpVariable("x%s_%s,%s"%(i,k,t),'Continuous') for i in range(customer_count)] for k in range(vehicle_count)]for t in range(kikan)]


#目的関数(1)
problem += pulp.lpSum(Ip[i][t]*h for i in range(customer_count) for t in range(kikan))

#制約（２）
for i in range(2):
    problem += pulp.lpSum(Q[i][k][t] for k in range(vehicle_count) for t in range(kikan)) <=2

#制約(3)
for i in range(1,customer_count):
    problem += (I[i][t] for t in syoumi) == (I[i][t] for t in range(kikan))

#制約(4)
#制約(5)
for i in range(1,customer_count):
    for t in range(kikan):
        if t >= m:
            problem += I[i][t-m+1] - pulp.lpSum(u[i][a] for a in range(t - m +2,t)) - pulp.lpSum(W[i][a] for a in range(t - m +2, t-1)) <=W[i][t]
        if t < m :
            problem += W[i][t] == 0

#制約（７）
for k in range(vehicle_count):
    for j in range(customer_count):
        problem += pulp.lpSum(x[i][j][k][t] if i != j else 0 for i in range(customer_count) for t in range(kikan)) - pulp.lpSum(x[j][i][k][t] if i != j else 0 for i in range(customer_count) for t in range(kikan)) == 0

#制約(8)
for j in range(customer_count):
    problem += pulp.lpSum(x[i][j][k] if i != j else 0 for i in range(customer_count-1) for k in range(vehicle_count-1)) <= 1

#制約(9)
for i in range(customer_count-1):
    for t in range(kikan):
        for k in range(vehicle_count-1):
            problem += pulp.lpSum(F[i][j][k][t] if i!= j else None for j in range(customer_count-1)) == pulp.lpSum(F[j][i][k][t] if i!= j else None for j in range(customer_count-1)) -Q[i][k][t]

#制約(10)
for i in range(customer_count-1):
    for j in range(customer_count):
        for k in range(vehicle_count):
            for t in range(kikan):
                problem += F[i][j][k][t] <= c * x[i][j][k][t]
#制約(21)
for i in range(customer_count):
    problem += pulp.lpSum(Q[i][k][s] for s in range(kikan) for k in range(vehicle_count)) - pulp.lpSum(W[i][s]for s in syoumi) >= pulp.lpSum(u[i][s] for s in syoumi)+np.sqrt(pulp.lpSum(u[i][s]**2 for s in range(kikan))) * C * Z
 
 
problem.solve()
print('目的関数値:', pulp.value(problem.objective))


KeyError: 0